In [1]:
import numpy as np
import pandas as pd
import os

import git
from git import RemoteProgress

from git import Repo
from datetime import datetime, date
import matplotlib.pyplot as plt
import seaborn as sns
from time import strptime
%matplotlib inline

In [2]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

In [3]:
# uncomment the correct repo you want the information from



# # apache poi 
# remote_link = "https://github.com/apache/poi"
# local_link = "repo/poi"
# # Uncomment to clone
# # Repo.clone_from(remote_link, local_link, progress=Progress())
# repo = Repo(local_link)
# vcc_commit = "eabb6a924be24abb879372d0bc967e0d316b2cf8"
# affected_files = ["src/ooxml/java/org/apache/poi/POIXMLTypeLoader.java",
#                   "src/ooxml/java/org/apache/poi/util/DocumentHelper.java"]




# cloudfoundry uua 
remote_link = "https://github.com/cloudfoundry/uaa"
local_link = "repo/uua"
# Uncomment to clone
# Repo.clone_from(remote_link, local_link, progress=Progress())
repo = Repo(local_link)
vcc_commit = "8dff169699d44c602c7b5942ba790e197117d627"
affected_files = ["server/src/main/java/org/cloudfoundry/identity/uaa/zone/GeneralIdentityZoneConfigurationValidator.java"]




# # apache struts 
# remote_link = "https://github.com/apache/struts"
# local_link = "repo/struts"
# # Uncomment to clone
# # Repo.clone_from(remote_link, local_link, progress=Progress())
# repo = Repo(local_link)
# vcc_commit = "ade5c99a072b19350c6cd1e7b1c79ffa031cdfa5"
# affected_files = ["core/src/main/java/com/opensymphony/xwork2/security/DefaultExcludedPatternsChecker.java",
#                   "xwork-core/src/test/java/com/opensymphony/xwork2/interceptor/ParametersInterceptorTest.java"]



In [4]:
def removeDuplicates(array):
    new_array = []
    for value in array:
        if value not in new_array:
            new_array.append(value)
    return new_array

In [5]:
# prints the commit message, 6a

show_data = repo.git.show("-s", vcc_commit).splitlines()
for line in show_data:
    print(line)


commit 8dff169699d44c602c7b5942ba790e197117d627
Author: cf-identity <cf-identity-eng@pivotal.io>
Date:   Mon Aug 28 17:10:16 2017 -0700

    Revert "Revert "Merge branch 'feature/banner-of-day-api' into develop""
    
    This reverts commit 55956f22c1de9234659105117811fc3da6677174.
    
    [finishes #150108579] https://www.pivotaltracker.com/story/show/150108579
    [finishes #150270498] https://www.pivotaltracker.com/story/show/150270498
    [finishes #150118804] https://www.pivotaltracker.com/story/show/150118804


In [6]:
# print the number of files changed, 6b

# retuns the files that have been changed
files = repo.git.whatchanged("-1", "--format=oneline", vcc_commit).splitlines()

# minus one as one is the commit message
files.pop(0)

print(str(len(files)) + " files changed")

9 files changed


In [7]:
# prints the number of unique directories, 6c

all_directories = []

for file in files:
    file_split = file.split()
    full_dir = file_split[5]
    directory = os.path.dirname(full_dir)
    all_directories.append(directory)
    
all_directories = removeDuplicates(all_directories)
print(str(len(all_directories)) + " directories changed")
    

5 directories changed


In [8]:
# prints the number of added/delete lines (inculding comments / blanks lines), 6d, 6e

lines_changed = repo.git.show("--pretty=tformat:", "--numstat", vcc_commit).splitlines()

total_lines_added = 0
total_lines_deleted = 0

for line in lines_changed:
    line_split = line.split()
    total_lines_added += int(line_split[0].strip())
    total_lines_deleted += int(line_split[1].strip())
    
print("Total deletions: " + str(total_lines_deleted))
print("Total additions: " + str(total_lines_added))

Total deletions: 5
Total additions: 528


In [9]:
def countNumberBlankComments(file, langauge):
    total_blank_additions = 0
    total_blank_deletions = 0
    
    if langauge == "Java":
        for line in file:
            if line[0] == "+":
                line = line[1:]
                line = line.strip()
                if line == "":
                    total_blank_additions += 1
                    continue
                if line[0] == "/" or line[0] == "*":
                    total_blank_additions += 1
                    
            if line[0] == "-":
                line = line[1:]
                line = line.strip()
                if line == "":
                    total_blank_deletions += 1
                    continue
                if line[0] == "/" or line[0] == "*":
                    total_blank_deletions += 1
    return total_blank_additions, total_blank_deletions

In [10]:
# prints the number of added/deleted lines (Excluding comments / blanks lines) 6f, 6g

total_blank = [0, 0]

# for each changed file
for file in files:
    # get the directory
    file_split = file.split()
    full_dir = file_split[5]
    
    #get the diff and find the blanks and comments     
    affected_file_diff = repo.git.diff(vcc_commit + "^" + ".." + vcc_commit, "--", full_dir).splitlines()
    this_total_blank = countNumberBlankComments(affected_file_diff, "Java")
    
    # add blanks and comments to total     
    total_blank = np.add(total_blank, this_total_blank)

# get the total from before and the found blank lines and remove them from total
total_blank_additions = total_lines_added - total_blank[0]
total_blank_deletions = total_lines_deleted - total_blank[1]

print("Total deletions (without comments/blanklines): " + str(total_blank_deletions))
print("Total additions (without comments/blanklines): " + str(total_blank_additions))

Total deletions (without comments/blanklines): 5
Total additions (without comments/blanklines): 458


In [11]:
def convertToDate(input_date):
    split = input_date.split()
    year = split[4]
    month = strptime(split[1],'%b').tm_mon
    day = split[2]
    return date(year = int(year), month = month, day = int(day))

In [12]:
# time difference between the fixing commit and the previous commit, 6h

for affected_file in affected_files:
    dates = repo.git.log("--format=%ad", vcc_commit, "--", affected_file).splitlines()
    if len(dates) <= 0:
        print("This file isn't in this commit: ", affected_file)
        continue
    date1_object = convertToDate(dates[0])
    if len(dates) < 2:
        print("no other changes have been made before")
    else:    
        date2_object = convertToDate(dates[1])
        total_days = abs((date1_object - date2_object).days)
        print("Total days since file", os.path.basename(affected_file), "changed:", total_days)

Total days since file GeneralIdentityZoneConfigurationValidator.java changed: 3


In [13]:
# get the number of times this file has been changed, 6i

for affected_file in affected_files:
    times = repo.git.log("--format=%ad", vcc_commit, "--", affected_file).splitlines()
    print("This file", os.path.basename(affected_file), "has been changed:", len(times), "times")

This file GeneralIdentityZoneConfigurationValidator.java has been changed: 13 times


In [14]:
# what devolpers have modifed this file? 6j
all_files_authors = []
for affected_file in affected_files:
    all_authors = repo.git.log("--format=%an", vcc_commit, "--", affected_file).splitlines()
    authors = removeDuplicates(all_authors)
    
    all_files_authors.extend(authors)
    all_files_authors = removeDuplicates(all_files_authors)
    
    print(os.path.basename(affected_file), "\n\tAuthors:")
    for author in authors:
        print("\t\t", author)
    print("")

GeneralIdentityZoneConfigurationValidator.java 
	Authors:
		 cf-identity
		 Henry Zhao
		 Bharath
		 Helen Chung
		 Filip Hanik
		 Bharath Sekar
		 Jeremy Coffield
		 Madhura Bhave



In [15]:
def getAuthorCommits(author_name, df):
    row = df[df["Author"].str.contains(author_name)]
    commits = row["Commit"].iloc[0]
    return int(commits)

In [16]:
# how many has each person commited

commit_authors = repo.git.shortlog("-sne", "--all").splitlines()

author_commits = []

for commit_author in commit_authors:
    commit, author = commit_author.split("\t")
    author_commits.append([int(commit), author])
    
df = pd.DataFrame(author_commits, columns=['Commit', 'Author'])

for author in all_files_authors:
    print(author, ":", getAuthorCommits(author, df), " Commits")



cf-identity : 271  Commits
Henry Zhao : 73  Commits
Bharath : 153  Commits
Helen Chung : 39  Commits
Filip Hanik : 2306  Commits
Bharath Sekar : 153  Commits
Jeremy Coffield : 217  Commits
Madhura Bhave : 209  Commits
